## Example of GraphCL

In [1]:
from sslgraph.utils import Encoder, EvalSemisupevised, EvalUnsupevised, get_dataset
from sslgraph.contrastive.model import GraphCL

### 1. Semi-supervised learning

#### Load dataset

In this example, we evaluate model on NCI1 dataset in the semi-supervised setting.

In [2]:
dataset, dataset_pretrain = get_dataset('DD', task='semisupervised')
feat_dim = dataset[0].x.shape[1]
embed_dim = 128

#### Define your encoder and contrastive model (GraphCL)

For semi-supervised setting, GraphCL uses ResGCN. 

Available augmentation includes: dropN, maskN, permE, subgraph, random[2-4].

In [3]:
encoder = Encoder(feat_dim, embed_dim, n_layers=3, gnn='resgcn')
graphcl = GraphCL(embed_dim, aug_1='subgraph', aug_2='dropN')

#### Define evaluatior instance

In this example, we use a label rate of 1%.

To setup configurations (num of epochs, learning rates, etc. for pretraining and finetuning), run


`evaluator.setup_train_config(batch_size = 128,
    p_optim = 'Adam', p_lr = 0.0001, p_weight_decay = 0, p_epoch = 100,
    f_optim = 'Adam', f_lr = 0.001, f_weight_decay = 0, f_epoch = 100)`


In [4]:
evaluator = EvalSemisupevised(dataset, dataset_pretrain, label_rate=0.1)

#### Perform evaluation

You can also perform evaluation with grid search on pre-training epoch and
learning rate by running
``
evaluator.grid_search(learning_model=graphcl, encoder=encoder, 
    p_lr_lst=[0.1,0.01,0.001,0.0001], p_epoch_lst=[20,40,60,80,100])
``

In [ ]:
evaluator.grid_search(learning_model=graphcl, encoder=encoder,
                      p_lr_lst=[0.01,0.001], p_epoch_lst=[20,40])

Pretraining: epoch 1:   0%|          | 0/20 [00:00<?, ?it/s]

To reproduce results in the paper, you may want to perform grid search and run evaluation for 5 times and take the average.